In [ ]:
import cv2
import numpy as np
import time
import logging

fname="detectron2.log"
#Create and configure logger 
logging.basicConfig(filename=fname, 
					format='%(asctime)s :: %(levelname)s :: %(message)s', 
					filemode='a+')

#Creating an object 
logger=logging.getLogger()

#Setting the threshold of logger to DEBUG 
logger.setLevel(logging.INFO)

# import some common detectron2 utilities
import torch, torchvision
import matplotlib.pyplot as plt
from detectron2.engine import DefaultPredictor
from detectron2.config import get_cfg
from detectron2.utils.visualizer import Visualizer
from detectron2.data import MetadataCatalog

class DETECTRON2():
    def __init__(self):
        # Create config
        self.cfg = get_cfg()
        self.cfg.merge_from_file("f/mask_rcnn_R_50_FPN_3x.yaml")
        self.cfg.SOLVER.IMS_PER_BATCH = 2
        self.cfg.SOLVER.BASE_LR = 0.01
        self.cfg.SOLVER.MAX_ITER =  1000   
        self.cfg.MODEL.ROI_HEADS.BATCH_SIZE_PER_IMAGE = 128   
        self.cfg.MODEL.ROI_HEADS.NUM_CLASSES = 2
        self.cfg.MODEL.WEIGHTS = "model.pth" # check in model weight file
        self.cfg.DATASETS.TEST = ("my_dataset_test", )
        self.cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.7   # set the testing threshold for this model
        self.cfg.MODEL.DEVICE='cpu'
        # set class names as seq inside list
        self.mtd=MetadataCatalog.get("my_dataset_test").set(thing_classes=['class1','class2'])
        self.predictor = DefaultPredictor(self.cfg)
        logger.info("weight files loaded")
        
    def predict(self,inputFilePath,fileSystemPath):
        # predict output
        mtd = self.mtd
        st = time.time()
        im = cv2.imread(inputFilePath)
        outputs = self.predictor(im)
        v = Visualizer(im[:, :, ::-1], metadata=self.mtd, scale=2.0)
        v = v.draw_instance_predictions(outputs["instances"].to("cpu"))
        
        ## write image only if defect is found & mark status of defect if found as yes or no
        # apply max confidence from image if any defect is found else set to 0
        logger.info('detectron time === '+str(time.time()-st))
#         print('detectron time === '+str(time.time()-st))
        try:
            confidences =int(max(outputs['instances'].scores)*100)
            cv2.imwrite(fileSystemPath,v.get_image()[:, :, ::-1])
            status= "Yes"
        except:
            status="No"
            confidences=0
        print(status,confidences)
        return [status,confidences]

In [ ]:
## prediction
model = DETECTRON2()
input_image_path='img.jpg'
output_image_path='predicted.jpg'
model.predict(input_image_path,output_image_path)
logger.info('-----------------------')
